In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
import keras.backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Floodnet'

In [ ]:
train_images_dir = os.path.join('train', 'images')
train_masks_dir = os.path.join('train', 'masks')
val_images_dir = os.path.join('val', 'images')
val_masks_dir = os.path.join('val', 'masks')


def count_files(directory):
    return len([file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))])


train_images_count = count_files(train_images_dir)
train_masks_count = count_files(train_masks_dir)
val_images_count = count_files(val_images_dir)
val_masks_count = count_files(val_masks_dir)


print(f"Number of files in {train_images_dir}: {train_images_count}")
print(f"Number of files in {train_masks_dir}: {train_masks_count}")
print(f"Number of files in {val_images_dir}: {val_images_count}")
print(f"Number of files in {val_masks_dir}: {val_masks_count}")


In [ ]:
train_folder = 'val'


images_folder = os.path.join(train_folder, 'images')
masks_folder = os.path.join(train_folder, 'masks')


image_files = sorted([f for f in os.listdir(images_folder) if f.endswith('.jpg')])
mask_files = sorted([f for f in os.listdir(masks_folder) if f.endswith('.png')])


idx = 90


img_path = os.path.join(images_folder, image_files[idx])
mask_path = os.path.join(masks_folder, mask_files[idx])

img = cv2.imread(img_path)
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title('Mask')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
train_images_dir = 'train/images'
train_masks_dir = 'train/masks'
val_images_dir = 'val/images'
val_masks_dir = 'val/masks'

In [ ]:


# Function to load and preprocess images and masks
def preprocess_images_and_masks(image_dir, mask_dir):
    images = []
    masks = []

    # Get list of filenames from directory
    file_names = os.listdir(image_dir)

    for filename in file_names:
        # Load image and mask
        image_path = os.path.join(image_dir, filename)
        mask_path = os.path.join(mask_dir, filename.replace('.jpg', '_lab.png'))  # Adjust mask filename

        image = load_img(image_path)   # Load and resize image
        mask = load_img(mask_path, color_mode='grayscale')  # Load and resize mask

        # Convert images and masks to numpy arrays and normalize
        image_array = img_to_array(image) / 255.0
        mask_array = img_to_array(mask)

        images.append(image_array)
        masks.append(mask_array)

    return np.array(images), np.array(masks)

In [ ]:
train_images, train_masks = preprocess_images_and_masks(train_images_dir, train_masks_dir)
np.save('train_images.npy', train_images)
np.save('train_masks.npy', train_masks)


val_images, val_masks = preprocess_images_and_masks(val_images_dir, val_masks_dir)
np.save('val_images.npy', val_images)
np.save('val_masks.npy', val_masks)

print(f"Training images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"Training masks shape: {train_masks.shape}, dtype: {train_masks.dtype}")
print(f"Validation images shape: {val_images.shape}, dtype: {val_images.dtype}")
print(f"Validation masks shape: {val_masks.shape}, dtype: {val_masks.dtype}")

In [ ]:
train_images_path = 'train_images.npy'
train_masks_path = 'train_masks.npy'

# Load numpy arrays
train_images = np.load(train_images_path)
train_masks = np.load(train_masks_path)

# Print shapes and types to verify
print(f"Loaded training images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"Loaded training masks shape: {train_masks.shape}, dtype: {train_masks.dtype}")

# Choose an example index to visualize
example_index = 200

# Display example image and mask
plt.figure(figsize=(10, 5))

# Plot the image
plt.subplot(1, 2, 1)
plt.imshow(train_images[example_index])
plt.title('Example Image')
plt.axis('off')

# Plot the mask
plt.subplot(1, 2, 2)
plt.imshow(train_masks[example_index].squeeze(), cmap='gray')
plt.title('Example Mask')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
val_images_path = 'val_images.npy'
val_masks_path = 'val_masks.npy'

# Load numpy arrays for validation
val_images = np.load(val_images_path)
val_masks = np.load(val_masks_path)

# Print shapes and types to verify
print(f"Loaded validation images shape: {val_images.shape}, dtype: {val_images.dtype}")
print(f"Loaded validation masks shape: {val_masks.shape}, dtype: {val_masks.dtype}")


In [ ]:
train_masks = train_masks.astype(np.uint8)
val_masks = val_masks.astype(np.uint8)

In [ ]:
train_masks_categorical = tf.keras.utils.to_categorical(train_masks, num_classes=10)
val_masks_categorical = tf.keras.utils.to_categorical(val_masks, num_classes=10)

In [ ]:
train_masks_categorical.shape

(2603, 512, 512, 10)

In [ ]:
val_masks_categorical.shape

(867, 512, 512, 10)

In [ ]:
loss1 = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
loss2 = tf.keras.losses.CategoricalFocalCrossentropy()

In [ ]:
def jaccard_loss(y_true, y_pred):
    smooth = 1e-10
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    sum_ = K.sum(y_true_f + y_pred_f)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return 1 - jac

In [ ]:
from keras.optimizers import Adam

In [ ]:

input_shape = (512, 512, 3)

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in vgg16.layers:
    layer.trainable = False


encoder_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3']
encoder_output = [vgg16.get_layer(layer_name).output for layer_name in encoder_layers]

# Decoder
# Contracting path (encoder)
inputs = vgg16.input
skips = encoder_output

# Expanding path (decoder)
x = skips[-1]  # Starting point of decoder
for i in range(1, len(skips)):
    x = Conv2DTranspose(512 // (2 ** i), (2, 2), strides=(2, 2), padding='same')(x)
    x = concatenate([skips[-i-1], x], axis=3)
    x = Conv2D(512 // (2 ** i), (3, 3), activation='relu', padding='same')(x)
    x = Dropout(0.5)(x)

# Final convolution layer (output layer)
num_classes = 10
outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

# Create the model
model = Model(inputs=[inputs], outputs=[outputs])

optimizer = Adam(learning_rate=0.0001)
# Compile the model
model.compile(optimizer='adam', loss=jaccard_loss, metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 512, 512, 64)         1792      ['input_4[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 512, 512, 64)         36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)         0         ['block1_conv2[0][0]']        
                                                                                            

In [ ]:
batch_size = 16
epochs = 2

# Train the model
history = model.fit(
    train_images,
    train_masks_categorical,
    validation_data=(val_images, val_masks_categorical),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

plt.figure(figsize=(12, 5))

# Plot training & validation loss values
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.title('Training and Validation Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
model.save('floodnet_model.h5')